In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
# Load the dataset using Pandas
data = pd.read_csv('diabetes.csv')

In [3]:
# For x: Extract out the dataset from all the rows (all samples) and all columns except last column (all features). 
# For y: Extract out the last column (which is the label)
# Convert both to numpy using the .values method
x = data.iloc[:,0:-1].values
y_string= list(data.iloc[:,-1])

In [4]:
# Lets have a look some samples from our data
print(x[:3])
print(y_string[:3])

[[  6.  148.   72.   35.    0.   33.6  50. ]
 [  1.   85.   66.   29.    0.   26.6  31. ]
 [  8.  183.   64.    0.    0.   23.3  32. ]]
['positive', 'negative', 'positive']


In [5]:
# Our neural network only understand numbers! So convert the string to labels
y_int = []
for string in y_string:
    if string == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [6]:
# Now convert to an array
y = np.array(y_int, dtype = 'float64')

In [7]:
# Feature Normalization. All features should have the same range of values (-1,1)
sc = StandardScaler()
x = sc.fit_transform(x)

In [8]:
# Now we convert the arrays to PyTorch tensors
x = torch.tensor(x)
# We add an extra dimension to convert this array to 2D for later use
y = torch.tensor(y).unsqueeze(1)

In [9]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [10]:
# build customized dataset, this class is inherited from torch.utils.data Dataset
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [11]:
dataset = Dataset(x,y)

In [12]:
len(dataset)

768

In [13]:
# Load the data to your dataloader for batch processing and shuffling
#can also import at the beginning "from torch.utils.data import DataLoader"
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=32,
                                           shuffle=True)

In [14]:
# check the data loader
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


In [15]:
# build the above network
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [17]:
# Create the network (an object of the Net class)
net = Model(x.shape[1])
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
criterion = torch.nn.BCELoss(size_average=True)   
# We will use SGD with momentum with a learning rate of 0.1 (weight, learning rate, momentum)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

In [18]:
# Train the network 
# in Pytorch, need use for loop to train
num_epochs = 200
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Feed Forward
        output = net(inputs)
        # Loss Calculation
        loss = criterion(output, labels)
        # Clear the gradient buffer (we don't want to accumulate gradients)
        optimizer.zero_grad()
        # Backpropagation 
        loss.backward()
        # Weight Update: w <-- w - lr * gradient, subtract lr*gradient from old weight
        optimizer.step()
        
    #Accuracy, this has fluctuation problem波动问题
    # Since we are using a sigmoid, we will need to perform some thresholding
    output = (output>0.5).float()
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))

Epoch 1/200, Loss: 0.679, Accuracy: 0.594
Epoch 2/200, Loss: 0.552, Accuracy: 0.719
Epoch 3/200, Loss: 0.571, Accuracy: 0.688
Epoch 4/200, Loss: 0.500, Accuracy: 0.812
Epoch 5/200, Loss: 0.407, Accuracy: 0.875
Epoch 6/200, Loss: 0.589, Accuracy: 0.719
Epoch 7/200, Loss: 0.441, Accuracy: 0.781
Epoch 8/200, Loss: 0.420, Accuracy: 0.875
Epoch 9/200, Loss: 0.451, Accuracy: 0.812
Epoch 10/200, Loss: 0.538, Accuracy: 0.781
Epoch 11/200, Loss: 0.649, Accuracy: 0.594
Epoch 12/200, Loss: 0.425, Accuracy: 0.812
Epoch 13/200, Loss: 0.364, Accuracy: 0.844
Epoch 14/200, Loss: 0.459, Accuracy: 0.719
Epoch 15/200, Loss: 0.454, Accuracy: 0.812
Epoch 16/200, Loss: 0.462, Accuracy: 0.781
Epoch 17/200, Loss: 0.502, Accuracy: 0.750
Epoch 18/200, Loss: 0.777, Accuracy: 0.562
Epoch 19/200, Loss: 0.556, Accuracy: 0.688
Epoch 20/200, Loss: 0.577, Accuracy: 0.719
Epoch 21/200, Loss: 0.413, Accuracy: 0.781
Epoch 22/200, Loss: 0.495, Accuracy: 0.750
Epoch 23/200, Loss: 0.370, Accuracy: 0.875
Epoch 24/200, Loss: 

Epoch 193/200, Loss: 0.256, Accuracy: 0.938
Epoch 194/200, Loss: 0.462, Accuracy: 0.719
Epoch 195/200, Loss: 0.402, Accuracy: 0.750
Epoch 196/200, Loss: 0.547, Accuracy: 0.781
Epoch 197/200, Loss: 0.447, Accuracy: 0.750
Epoch 198/200, Loss: 0.299, Accuracy: 0.875
Epoch 199/200, Loss: 0.442, Accuracy: 0.781
Epoch 200/200, Loss: 0.381, Accuracy: 0.844
